In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd

# dataset
data = pd.read_csv('../data/weather/weather.csv')

# the list of unique countries
countries = data['country'].unique()

# create a dash app in python
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Weather Data"),
    html.Label("Select a country:"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in countries],
        value='Finland'
    ),
    html.Label("Select a date range:"),
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date='2016-01-01',
        end_date='2020-12-31'
    ),
    dcc.Checklist(
        id='weather-data-selection',
        options=[
            {'label': 'Temperature', 'value': 'temp_mean'},
            {'label': 'Rainfall', 'value': 'rain'},
            {'label': 'Snowfall', 'value': 'snow'}
        ],
        value=['temp_mean', 'rain', 'snow']
    ),
    dcc.Graph(id='weather-plot'),
    html.Div(id='statistics')  # container for statistics
])

@app.callback(
    [Output('weather-plot', 'figure'),
     Output('statistics', 'children')],
    [Input('country-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date'),
     Input('weather-data-selection', 'value')]
)
def update_plot_and_statistics(selected_country, start_date, end_date, selected_data):
    # filter the data for the selected country and date range
    filtered_data = data[(data['country'] == selected_country) & (data['date'] >= start_date) & (data['date'] <= end_date)]

    fig = make_subplots(specs=[[{"secondary_y": True}]])  # Create subplots with a secondary y-axis

    for data_point in selected_data:
        if data_point == 'temp_mean':
            fig.add_trace(go.Scatter(x=filtered_data['date'], y=filtered_data[data_point], mode='lines', name='Temperature'))
        else:
            fig.add_trace(go.Scatter(x=filtered_data['date'], y=filtered_data[data_point], mode='lines', name=data_point), secondary_y=True)

    fig.update_layout(
        title=f'Weather Data in {selected_country}',
        xaxis_title="Date",
        xaxis_rangeslider_visible=True
    )

    statistics_text = html.Div([
        html.H2("Statistics"),
    ])
    for data_point in selected_data:
        mean_value = filtered_data[data_point].mean()
        max_value = filtered_data[data_point].max()
        min_value = filtered_data[data_point].min()
        statistics_text.children.append(html.P(f"Mean {data_point}: {mean_value:.2f}"))
        statistics_text.children.append(html.P(f"Max {data_point}: {max_value:.2f}"))
        statistics_text.children.append(html.P(f"Min {data_point}: {min_value:.2f}"))

    return fig, statistics_text

if __name__ == '__main__':
    app.run_server(debug=True, port=8053)


In [2]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.5 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
